In [10]:
from google.colab import files
from google.colab import drive
drive.mount()

In [9]:
files.upload()

KeyboardInterrupt: 

In [5]:
!pip install ngrok
!pip install pyngrok
!pip install flask_ngrok
!pip install pytorch_lightning
!pip install model_loader

ERROR: Could not find a version that satisfies the requirement model_loader (from versions: none)
ERROR: No matching distribution found for model_loader


In [6]:
import model_loader
import pipeline
from PIL import Image
from pathlib import Path
from transformers import CLIPTokenizer
import torch
from flask import Flask, render_template, request, send_file
from io import BytesIO
import os
import numpy as np
import ngrok
import threading
from flask import Flask
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
import pytorch_lightning

ModuleNotFoundError: No module named 'model_loader'

In [2]:
# Define the path for the generated images folder
GENERATED_IMAGES_FOLDER = os.path.join('/content/generated_images')

# Ensure the folder exists, create if not
os.makedirs(GENERATED_IMAGES_FOLDER, exist_ok=True)

DEVICE = "cpu"

ALLOW_CUDA = True
ALLOW_MPS = False

if torch.cuda.is_available() and ALLOW_CUDA:
    DEVICE = "cuda"
elif (torch.has_mps or torch.backends.mps.is_available()) and ALLOW_MPS:
    DEVICE = "mps"
print(f"Using device: {DEVICE}")

tokenizer = CLIPTokenizer("/content/drive/MyDrive/Capstone/data/vocab.json", merges_file="/content/drive/MyDrive/Capstone/data/merges.txt")
model_file = "/content/drive/MyDrive/Capstone/data/v1-5-pruned-emaonly.ckpt"
models = model_loader.preload_models_from_standard_weights(model_file, DEVICE)

## Move models to GPU if available
if DEVICE == "cuda":
    for model in models.values():
        model.to("cuda")

## TEXT TO IMAGE

# prompt = "A dog with sunglasses, wearing comfy hat, looking at camera, highly detailed, ultra sharp, cinematic, 100mm lens, 8k resolution."
prompt = "Mickey Mouse siting on bike"
uncond_prompt = ""  # Also known as negative prompt
do_cfg = True
cfg_scale = 8  # min: 1, max: 14

## IMAGE TO IMAGE

input_image = None
# Comment to disable image to image
image_path = "../images/dog.jpg"
# input_image = Image.open(image_path)
# Higher values means more noise will be added to the input image, so the result will further from the input image.
# Lower values means less noise is added to the input image, so output will be closer to the input image.
strength = 0.9

## SAMPLER

sampler = "ddpm"
num_inference_steps = 50
seed = 42

output_image = pipeline.generate(
    prompt=prompt,
    uncond_prompt=uncond_prompt,
    input_image=input_image,
    strength=strength,
    do_cfg=do_cfg,
    cfg_scale=cfg_scale,
    sampler_name=sampler,
    n_inference_steps=num_inference_steps,
    seed=seed,
    models=models,
    device=DEVICE,
    idle_device="cpu",
    tokenizer=tokenizer,
)

# Convert the NumPy array to a PIL Image object
output_image_pil = Image.fromarray(np.uint8(output_image))

# Save the output image to the generated_images folder
output_image_path = os.path.join(GENERATED_IMAGES_FOLDER, 'generated_image.png')
output_image_pil.save(output_image_path)

# Combine the input image and the output image into a single image.
Image.fromarray(output_image)

NameError: name 'os' is not defined

In [ ]:
app = Flask(__name__)

# Define the path for the generated images folder
GENERATED_IMAGES_FOLDER = os.path.join(app.root_path, 'generated_images')

# Ensure the folder exists, create if not
os.makedirs(GENERATED_IMAGES_FOLDER, exist_ok=True)

DEVICE = "cpu"

ALLOW_CUDA = False
ALLOW_MPS = False

if torch.cuda.is_available() and ALLOW_CUDA:
    DEVICE = "cuda"
elif (torch.has_mps or torch.backends.mps.is_available()) and ALLOW_MPS:
    DEVICE = "mps"
print(f"Using device: {DEVICE}")

tokenizer = CLIPTokenizer("/content/drive/MyDrive/Capstone/data/vocab.json", merges_file="/content/drive/MyDrive/Capstone/data/merges.txt")
model_file = "/content/drive/MyDrive/Capstone/data/v1-5-pruned-emaonly.ckpt"
models = model_loader.preload_models_from_standard_weights(model_file, DEVICE)


"""
## TEXT TO IMAGE

# prompt = "A dog with sunglasses, wearing comfy hat, looking at camera, highly detailed, ultra sharp, cinematic, 100mm lens, 8k resolution."
prompt = "Generate a calming painting consisting of doraemon , mandala in background with a turquoise theme"
uncond_prompt = ""  # Also known as negative prompt
do_cfg = True
cfg_scale = 8  # min: 1, max: 14

## IMAGE TO IMAGE

input_image = None
# Comment to disable image to image
image_path = "../images/dog.jpg"
# input_image = Image.open(image_path)
# Higher values means more noise will be added to the input image, so the result will further from the input image.
# Lower values means less noise is added to the input image, so output will be closer to the input image.
strength = 0.9

## SAMPLER

sampler = "ddpm"
num_inference_steps = 50
seed = 42

output_image = pipeline.generate(
    prompt=prompt,
    uncond_prompt=uncond_prompt,
    input_image=input_image,
    strength=strength,
    do_cfg=do_cfg,
    cfg_scale=cfg_scale,
    sampler_name=sampler,
    n_inference_steps=num_inference_steps,
    seed=seed,
    models=models,
    device=DEVICE,
    idle_device="cpu",
    tokenizer=tokenizer,
)

# Combine the input image and the output image into a single image.
Image.fromarray(output_image)

"""

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/generate', methods=['POST'])
def generate_image():
    prompt = request.form['prompt']
    uncond_prompt = request.form['uncond_prompt']

    # Handle file upload
    input_image = None
    if 'input_image' in request.files:
        input_image = Image.open(request.files['input_image'])

    strength = 0.9
    do_cfg = True
    cfg_scale = 8
    sampler = "ddpm"
    num_inference_steps = 50
    seed = 42

    output_image = pipeline.generate(
        prompt=prompt,
        uncond_prompt=uncond_prompt,
        input_image=input_image,
        strength=strength,
        do_cfg=do_cfg,
        cfg_scale=cfg_scale,
        sampler_name=sampler,
        n_inference_steps=num_inference_steps,
        seed=seed,
        models=models,
        device=DEVICE,
        idle_device="cpu",
        tokenizer=tokenizer,
    )

     # Convert the NumPy array to a PIL Image object
    output_image_pil = Image.fromarray(np.uint8(output_image))

    # Save the output image to the generated_images folder
    output_image_path = os.path.join(GENERATED_IMAGES_FOLDER, 'generated_image.png')
    output_image_pil.save(output_image_path)

    # Save the output image to a BytesIO object
    output_buffer = BytesIO()
    output_image_pil.save(output_buffer, format='PNG')
    output_buffer.seek(0)

    # Return the output image as a file to be displayed in the browser
    return send_file(output_buffer, mimetype='image/png')
    """
    image_path = 'image.jpg'
    return send_file(image_path, mimetype='image/jpg')
    """
if __name__ == '__main__':
    app.run(host='0.0.0.0')

<ipython-input-2-f360f2a8533c>:16: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  elif (torch.has_mps or torch.backends.mps.is_available()) and ALLOW_MPS:


Using device: cpu
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
os.environ["FLASK_ENV"] = "development"
app = Flask(__name__)
port = 80

ngrok.set_auth_token("2g2pkhP9Z4Sm3TOau6EHjYvCSLV_4UikzqQpXSH1wPTFeYPVq")

public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

app.config["BASE_URL"] = public_url

@app.route("/", methods=['POST'])
def index():
    image_path = 'image.jpg'
    return send_file(image_path, mimetype='image/jpg')

if __name__ == '__main__':
    app.run(host='0.0.0.0')

'FLASK_ENV' is deprecated and will not be used in Flask 2.3. Use 'FLASK_DEBUG' instead.


 * ngrok tunnel "https://ba38-34-143-208-215.ngrok-free.app" -> "http://127.0.0.1:80"
 * Serving Flask app '__main__'
 * Debug mode: on


'FLASK_ENV' is deprecated and will not be used in Flask 2.3. Use 'FLASK_DEBUG' instead.
'FLASK_ENV' is deprecated and will not be used in Flask 2.3. Use 'FLASK_DEBUG' instead.
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
from flask import Flask, jsonify, request
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def hello():
 return 'Hello World!! from Google Colab'

if __name__ == '__main__':
    app.run(host='0.0.0.0')

'FLASK_ENV' is deprecated and will not be used in Flask 2.3. Use 'FLASK_DEBUG' instead.


TypeError: run_with_ngrok.<locals>.new_run() got an unexpected keyword argument 'host'

In [ ]:
!curl 127.0.0.1:80

curl: (7) Failed to connect to 127.0.0.1 port 80 after 0 ms: Connection refused


In [ ]:
!curl -v http://localhost:80/

*   Trying 127.0.0.1:80...
* connect to 127.0.0.1 port 80 failed: Connection refused
*   Trying ::1:80...
* Immediate connect fail for ::1: Cannot assign requested address
* Failed to connect to localhost port 80 after 0 ms: Connection refused
* Closing connection 0
curl: (7) Failed to connect to localhost port 80 after 0 ms: Connection refused


In [ ]:
import firebase_admin
from firebase_admin import credentials, db

In [ ]:
cred = credentials.Certificate("/content/generative-therapy-app-firebase-adminsdk-2fdhx-e5f50b1dbd.json")
firebase_admin.initialize_app(cred)

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [ ]:
# Get a reference to the database service
ref = db.reference('/')

# Retrieve data from the database
data = ref.get()

# Print the retrieved data
print(data)

ValueError: Invalid database URL: "None". Database URL must be a non-empty URL string.